In [1]:
import pandas as pd
import numpy as np

In [2]:
#Visualisation
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 1000)

# Format année
def format_annee(code_annee):

    if code_annee == '2011-2013':
        return 2012

    elif code_annee == '2012-2014':
        return 2013

    elif code_annee == '2013-2015':
        return 2014

    elif code_annee == '2014-2016':
        return 2015

    elif code_annee == '2015-2017':
        return 2016

    elif code_annee == '2016-2018':
        return 2017

In [3]:
# Population mondiale par pays
df_pop = pd.read_csv("/Users/teilo/Desktop/data/fr_population.csv")[['Zone', 'Valeur']]
df_pop.columns = ['Zone', 'pop_total']

# Echelle
df_pop.fillna(0, inplace=True)
df_pop['pop_total'] = df_pop['pop_total'] * 1000

df_pop.head(5)
pop_total = df_pop['pop_total'].sum()
print(f"""Population du Monde en 2013: {pop_total:,}""")
display(df_pop[df_pop['Zone'].str.contains('Chine')])

# Suppression de la Chine
df_pop = df_pop[df_pop.Zone != 'Chine'].copy()
pop_total = df_pop['pop_total'].sum()

print(f"""Population du Monde en 2013: {pop_total:,}""")

Population du Monde en 2013: 8,413,993,000


,Zone,pop_total
33,Chine,1416667000
34,Chine - RAS de Hong-Kong,7204000
35,Chine - RAS de Macao,566000
36,"Chine, continentale",1385567000
37,"Chine, Taiwan Province de",23330000


Population du Monde en 2013: 6,997,326,000


In [4]:
# Part 2

# Colonne pour Bilans alimentaires
ba_columns = ['Zone', 'Code zone', 'Élément', 'Code Produit', 'Produit', 'Année', 'Valeur']
sa_columns = ['Zone', 'Code zone', 'Produit', 'Année', 'Valeur']
# Création des DataFrames

# Céreales
df_animals = pd.read_csv("/Users/teilo/Desktop/data/fr_animaux.csv")[ba_columns]
df_animals['Origine'] = 'animal'
display(df_animals.head(7))

# Végetaux
df_vegetables = pd.read_csv("/Users/teilo/Desktop/data/fr_vegetaux.csv")[ba_columns]
df_vegetables['Origine'] = 'vegetal'
df_vegetables.head(5)

# Concatenation
main_df = pd.concat([df_vegetables, df_animals])

main_df = pd.pivot_table(main_df, values='Valeur', index=['Zone', 'Code zone', 'Produit', 'Origine'], columns='Élément')
main_df = main_df.rename_axis(None, axis=1)
main_df.reset_index(inplace=True)

,Zone,Code zone,Élément,Code Produit,Produit,Année,Valeur,Origine
0,Afghanistan,2,Production,2731,Viande de Bovins,2013,134.00,animal
1,Afghanistan,2,Importations - Quantité,2731,Viande de Bovins,2013,6.00,animal
2,Afghanistan,2,Disponibilité intérieure,2731,Viande de Bovins,2013,140.00,animal
3,Afghanistan,2,Nourriture,2731,Viande de Bovins,2013,140.00,animal
4,Afghanistan,2,Disponibilité alimentaire en quantité (kg/pers...,2731,Viande de Bovins,2013,4.59,animal
5,Afghanistan,2,Disponibilité alimentaire (Kcal/personne/jour),2731,Viande de Bovins,2013,27.00,animal
6,Afghanistan,2,Disponibilité de protéines en quantité (g/pers...,2731,Viande de Bovins,2013,1.89,animal


In [5]:
# Renommés
main_df.rename(columns={'Code zone' : 'code', 'Code Produit' : 'code_produit', 'Aliments pour animaux' : 'alim_anim', 'Autres utilisations (non alimentaire)' : 'autres_usages', 'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_pers_jour', 'Disponibilité alimentaire en quantité (kg/personne/an)' : 'dispo_alim_quant_kg_pers_an', 'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_gras_g_pers_jour', 'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot_g_pers_jour', 'Disponibilité intérieure' : 'dispo_interieur', 'Exportations - Quantité' : 'quant_exportation', 'Importations - Quantité' : 'quant_importation', 'Variation de stock' : 'variation_stocks'}, inplace=True)

# Rassemblé sur le Zone
main_df = pd.merge(main_df,df_pop[['Zone','pop_total']], how='inner', on='Zone')

# Suppression des redondance
main_df.drop_duplicates()

,Zone,code,Produit,Origine,alim_anim,autres_usages,dispo_alim_kcal_pers_jour,dispo_alim_quant_kg_pers_an,dispo_gras_g_pers_jour,dispo_prot_g_pers_jour,dispo_interieur,quant_exportation,quant_importation,Nourriture,Pertes,Production,Semences,Traitement,variation_stocks,pop_total
0,Afghanistan,2,Abats Comestible,animal,NaN,NaN,5.0,1.72,0.20,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN,30552000
1,Afghanistan,2,"Agrumes, Autres",vegetal,NaN,NaN,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN,30552000
2,Afghanistan,2,Aliments pour enfants,vegetal,NaN,NaN,1.0,0.06,0.01,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,30552000
3,Afghanistan,2,Ananas,vegetal,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,30552000
4,Afghanistan,2,Bananes,vegetal,NaN,NaN,4.0,2.70,0.02,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN,30552000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,25,Viande de Suides,animal,NaN,NaN,45.0,4.70,4.28,1.41,3.0,NaN,0.0,3.0,NaN,2.0,NaN,NaN,NaN,561000
15601,Îles Salomon,25,Viande de Volailles,animal,NaN,NaN,11.0,3.34,0.69,1.14,2.0,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,561000
15602,Îles Salomon,25,"Viande, Autre",animal,NaN,NaN,0.0,0.06,NaN,0.04,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,561000
15603,Îles Salomon,25,Vin,vegetal,NaN,NaN,0.0,0.07,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,561000


In [6]:
# Remise au kilo

main_df['quant_exportation'] = main_df['quant_exportation'] * 1000000
main_df['quant_importation'] = main_df['quant_importation'] * 1000000
main_df['autres_usages'] = main_df['autres_usages'] * 1000000

main_df['Semences'] = main_df['Semences'] * 1000000
main_df['alim_anim'] = main_df['alim_anim'] * 1000000
main_df['Pertes'] = main_df['Pertes'] * 1000000
main_df['Production'] = main_df['Production'] * 1000000
main_df['Traitement'] = main_df['Traitement'] * 1000000
main_df['variation_stocks'] = main_df['variation_stocks'] * 1000000
main_df['dispo_interieur'] = main_df['dispo_interieur'] * 1000000
main_df['Nourriture'] = main_df['Nourriture'] * 1000000


main_df.head(10)

,Zone,code,Produit,Origine,alim_anim,autres_usages,dispo_alim_kcal_pers_jour,dispo_alim_quant_kg_pers_an,dispo_gras_g_pers_jour,dispo_prot_g_pers_jour,dispo_interieur,quant_exportation,quant_importation,Nourriture,Pertes,Production,Semences,Traitement,variation_stocks,pop_total
0,Afghanistan,2,Abats Comestible,animal,NaN,NaN,5.0,1.72,0.20,0.77,5.300000e+07,NaN,NaN,5.300000e+07,NaN,5.300000e+07,NaN,NaN,NaN,30552000
1,Afghanistan,2,"Agrumes, Autres",vegetal,NaN,NaN,1.0,1.29,0.01,0.02,4.100000e+07,2000000.0,4.000000e+07,3.900000e+07,2000000.0,3.000000e+06,NaN,NaN,NaN,30552000
2,Afghanistan,2,Aliments pour enfants,vegetal,NaN,NaN,1.0,0.06,0.01,0.03,2.000000e+06,NaN,2.000000e+06,2.000000e+06,NaN,NaN,NaN,NaN,NaN,30552000
3,Afghanistan,2,Ananas,vegetal,NaN,NaN,0.0,0.00,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,30552000
4,Afghanistan,2,Bananes,vegetal,NaN,NaN,4.0,2.70,0.02,0.05,8.200000e+07,NaN,8.200000e+07,8.200000e+07,NaN,NaN,NaN,NaN,NaN,30552000
5,Afghanistan,2,"Beurre, Ghee",animal,NaN,NaN,23.0,1.17,2.61,0.03,3.600000e+07,NaN,0.000000e+00,3.600000e+07,NaN,3.600000e+07,NaN,NaN,NaN,30552000
6,Afghanistan,2,Bière,vegetal,NaN,NaN,0.0,0.09,NaN,0.00,3.000000e+06,NaN,3.000000e+06,3.000000e+06,NaN,NaN,NaN,NaN,NaN,30552000
7,Afghanistan,2,Blé,vegetal,NaN,NaN,1369.0,160.23,4.69,36.91,5.992000e+09,NaN,1.173000e+09,4.895000e+09,775000000.0,5.169000e+09,322000000.0,NaN,-350000000.0,30552000
8,Afghanistan,2,Boissons Alcooliques,vegetal,NaN,NaN,0.0,0.00,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,30552000
9,Afghanistan,2,Café,vegetal,NaN,NaN,0.0,0.00,NaN,0.00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,30552000


In [7]:
# Réponse Part 2 ---------------------------------------------------------------

# France
fr_df = main_df[main_df['Zone'] == 'France']

# Blé
ble_fr_df = fr_df[fr_df['Produit'] == 'Blé']

ble_fr_df.head(5)

,Zone,code,Produit,Origine,alim_anim,autres_usages,dispo_alim_kcal_pers_jour,dispo_alim_quant_kg_pers_an,dispo_gras_g_pers_jour,dispo_prot_g_pers_jour,dispo_interieur,quant_exportation,quant_importation,Nourriture,Pertes,Production,Semences,Traitement,variation_stocks,pop_total
4851,France,68,Blé,vegetal,7.822000e+09,2.824000e+09,811.0,108.43,4.2,25.99,2.029800e+10,2.150200e+10,2.055000e+09,6.971000e+09,358000000.0,3.861400e+10,748000000.0,1.575000e+09,1.131000e+09,64291000


In [8]:
#Recupération des valeurs
production = ble_fr_df['Production'].values
importation = ble_fr_df['quant_importation'].values
exportation = ble_fr_df['quant_exportation'].values
variation_stock = ble_fr_df['variation_stocks'].values
production = ble_fr_df['Production'].values
semence = ble_fr_df['Semences'].values
nourriture = ble_fr_df['Nourriture'].values
aliments_animaux = ble_fr_df['alim_anim'].values
autres_usages = ble_fr_df['autres_usages'].values
perte = ble_fr_df['Pertes'].values
dispo_interieur = ble_fr_df['dispo_interieur'].values
traitement = ble_fr_df['Traitement'].values

# Résultat
print(f"""Production : {production}
Importations : {importation}
Exportations : {exportation}
Stock Variation : {variation_stock}
Disponibilité Intérieur : {dispo_interieur}
Semences : {semence}
Pertes : {perte}
Nourriture : {nourriture}
Aliments pour Animaux : {aliments_animaux}
Traitement : {traitement}
Autres Usages : {autres_usages}""")


term1 = production + importation - exportation + variation_stock
term2 = dispo_interieur
term3 = semence + nourriture + traitement + aliments_animaux + autres_usages + perte


print(f"""D'après la FAO les disponibilité intérieur peuvent etre calculer de plusieurs façon soit : production + importation - exportation + variation_stock = {term1}""")
print(f"""ou bien semence + nourriture + traitement + aliments_animaux + autres_usages + perte = {term3} ainsi d'après le document les disponibilité intérieur = {term3} """)

Production : [3.8614e+10]
Importations : [2.055e+09]
Exportations : [2.1502e+10]
Stock Variation : [1.131e+09]
Disponibilité Intérieur : [2.0298e+10]
Semences : [7.48e+08]
Pertes : [3.58e+08]
Nourriture : [6.971e+09]
Aliments pour Animaux : [7.822e+09]
Traitement : [1.575e+09]
Autres Usages : [2.824e+09]
D'après la FAO les disponibilité intérieur peuvent etre calculer de plusieurs façon soit : production + importation - exportation + variation_stock = [2.0298e+10]
ou bien semence + nourriture + traitement + aliments_animaux + autres_usages + perte = [2.0298e+10] ainsi d'après le document les disponibilité intérieur = [2.0298e+10] 


In [9]:
# Partie 3 ---------------------------------------------------------------------
# Calculer (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

#Calcul des disponibilité kcal et proteine alimentaire
main_df['total_alim_dispo'] = main_df['dispo_alim_kcal_pers_jour'] * 365 * main_df['pop_total']
main_df['total_proteine_dispo'] = main_df['dispo_prot_g_pers_jour'] *0.001 * main_df['pop_total'] * 365

# > Diviser les g par 1000 pour atteindre la valeur en kg.
main_df.head()

,Zone,code,Produit,Origine,alim_anim,autres_usages,dispo_alim_kcal_pers_jour,dispo_alim_quant_kg_pers_an,dispo_gras_g_pers_jour,dispo_prot_g_pers_jour,dispo_interieur,quant_exportation,quant_importation,Nourriture,Pertes,Production,Semences,Traitement,variation_stocks,pop_total,total_alim_dispo,total_proteine_dispo
0,Afghanistan,2,Abats Comestible,animal,NaN,NaN,5.0,1.72,0.20,0.77,53000000.0,NaN,NaN,53000000.0,NaN,53000000.0,NaN,NaN,NaN,30552000,5.575740e+10,8586639.6
1,Afghanistan,2,"Agrumes, Autres",vegetal,NaN,NaN,1.0,1.29,0.01,0.02,41000000.0,2000000.0,40000000.0,39000000.0,2000000.0,3000000.0,NaN,NaN,NaN,30552000,1.115148e+10,223029.6
2,Afghanistan,2,Aliments pour enfants,vegetal,NaN,NaN,1.0,0.06,0.01,0.03,2000000.0,NaN,2000000.0,2000000.0,NaN,NaN,NaN,NaN,NaN,30552000,1.115148e+10,334544.4
3,Afghanistan,2,Ananas,vegetal,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,30552000,0.000000e+00,NaN
4,Afghanistan,2,Bananes,vegetal,NaN,NaN,4.0,2.70,0.02,0.05,82000000.0,NaN,82000000.0,82000000.0,NaN,NaN,NaN,NaN,NaN,30552000,4.460592e+10,557574.0


In [10]:
# Question 5 
ratio = main_df[['Produit', 'total_alim_dispo', 'total_proteine_dispo', 'Nourriture']].copy()

#replacement des 0 par des NaN afin d'éviter les données érronnées
ratio.replace(0, np.nan, inplace=True)

# ratios
ratio['ratio_kcal'] = ratio['total_alim_dispo']/ratio['Nourriture']
ratio['ratio_kgprot'] = ratio['total_proteine_dispo']/ratio['Nourriture']

# ratio par catégorie
ratio = ratio[['Produit', 'ratio_kcal', 'ratio_kgprot']].groupby('Produit').mean()
ratio.reset_index(inplace=True)

ratio.loc[ratio['Produit'] == 'Oeufs', :]

,Produit,ratio_kcal,ratio_kgprot
58,Oeufs,1345.300386,0.105592


In [11]:
ratio.sort_values('ratio_kcal', ascending=False).head(20)

,Produit,ratio_kcal,ratio_kgprot
45,Huiles de Foie de Poisso,9982.932500,NaN
43,Huile de Sésame,9433.744729,NaN
42,Huile de Son de Riz,9361.549883,0.006927
46,Huiles de Poissons,9003.715504,NaN
34,Huile d'Arachide,8935.765811,0.010132
35,Huile d'Olive,8931.223447,NaN
38,Huile de Germe de Maïs,8871.762566,0.007495
32,Huil Plantes Oleif Autr,8848.222893,0.011047
40,Huile de Palmistes,8789.540277,0.006703
33,Huile Graines de Coton,8750.320136,0.005643


In [12]:
ratio.sort_values('ratio_kgprot', ascending=False).head(20)

,Produit,ratio_kcal,ratio_kgprot
27,Graines Colza/Moutarde,4150.299049,0.285365
81,Soja,3287.966903,0.285331
6,Arachides Decortiquees,5702.900100,0.251466
70,Pois,3475.044111,0.223244
50,Légumineuses Autres,3416.965957,0.219716
31,Haricots,3395.683518,0.215452
0,Abats Comestible,1129.258163,0.175610
87,Sésame,5796.120413,0.175040
95,"Viande, Autre",1295.626878,0.174024
69,"Plantes Oleiferes, Autre",4350.089305,0.170544


In [13]:
# Partie 6 ----------------------------------------------------------------------------
# Calculer la disponibilité intérieure en kcal des produits végétaux

# création de la table
vegetal = main_df.loc[main_df['Origine'] == 'vegetal', :]

# conversions des données dispo int en kcal et prot
vegetal = pd.merge(vegetal, ratio, on='Produit', how='left')
vegetal['dispo_int_kcal'] = vegetal['dispo_interieur'] * vegetal['ratio_kcal']

vegetal['dispo_int_prot'] = vegetal['dispo_interieur'] * vegetal['ratio_kgprot']

print('Disponibilité totale en kcal =', vegetal['dispo_int_kcal'].sum(), 'kcal')
print('Disponibilité totale en proteines =', vegetal['dispo_int_prot'].sum(), 'proteines')

Disponibilité totale en kcal = 1.3764013420014768e+16 kcal
Disponibilité totale en proteines = 348844629023.2972 proteines


In [14]:
# Partie 7 ----------------------------------------------------------------------------
# Nombre d'humains nourris grâce à la production mondiale végétale, kcal, prot et %
# Selon OMS un humain à besoin en moyenne de 2225 kcal/jour
# Selon WWF les besoins annuelle en proteine sont de 49,6g par jour (62kg * 0,8g)

#Besoin kcal humains
bkcal = 2225 * 365
bprot = 49.6 * 365 * 0.001

# Population mondiale
pop_total = df_pop['pop_total'].sum()
print(f"""Population totale : {pop_total:,}""")

# kcal / humain
kcal_humain = vegetal['dispo_int_kcal'].sum()/bkcal
kcal_pourcent = kcal_humain / pop_total

# prot / humain
prot_humain = vegetal['dispo_int_prot'].sum()/bprot
prot_pourcent = prot_humain / pop_total

print('Grâce à la production végétale mondiale nous pourrions nourrir', round(kcal_humain),'humains soit', round(kcal_pourcent*100, 2),'% de la population')
print('Grâce à la production végétale mondiale nous pourrions nourrir', round(prot_humain),'humains soit', round(prot_pourcent*100, 2),'% de la population')

Population totale : 6,997,326,000
Grâce à la production végétale mondiale nous pourrions nourrir 16948146431 humains soit 242.21 % de la population
Grâce à la production végétale mondiale nous pourrions nourrir 19268925598 humains soit 275.38 % de la population


In [15]:
# Partie 8 -------------------------------------------------------------------------------------------
# Part de la population que l'on pourrait nourrir avec pertes et nourriture pour animaux en plus

# Création du DF
vegetal['somme_totale'] = vegetal[['Nourriture', 'alim_anim', 'Pertes']].sum(axis=1)

# Calculs des valeurs
kcal_pour_humain = (vegetal['somme_totale'] * vegetal['ratio_kcal']).sum() /bkcal
prot_pour_humain = (vegetal['somme_totale'] * vegetal['ratio_kgprot']).sum() /bprot


print('Grâce à la production végétale mondiale nous pourrions nourrir', round(kcal_pour_humain),'humains soit', round(kcal_pour_humain*100/pop_total, 2),'% de la population')
print('Grâce à la production végétale mondiale nous pourrions nourrir', round(prot_pour_humain),'humains soit', round(prot_pour_humain*100/pop_total, 2),'% de la population')


Grâce à la production végétale mondiale nous pourrions nourrir 11701135891 humains soit 167.22 % de la population
Grâce à la production végétale mondiale nous pourrions nourrir 11832353837 humains soit 169.1 % de la population


In [16]:
#Parie 9 -----------------------------------------------------------------------------------
# Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ?

# Calculs des valeurs
temp_kcal = main_df['total_alim_dispo'].sum()/bkcal
temp_prot = main_df['total_proteine_dispo'].sum()/bprot


print('Grâce à la production végétale mondiale nous pourrions nourrir', round(temp_kcal),'humains soit', round(temp_kcal*100/pop_total, 2),'% de la population')
print('Grâce à la production végétale mondiale nous pourrions nourrir', round(temp_prot),'humains soit', round(temp_prot*100/pop_total, 2),'% de la population')



Grâce à la production végétale mondiale nous pourrions nourrir 9067426098 humains soit 129.58 % de la population
Grâce à la production végétale mondiale nous pourrions nourrir 11448253429 humains soit 163.61 % de la population


In [17]:
#Partie 10 ------------------------------------------------------------------------------------------
# Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

#Détermination des données concernant la sous-nutrition
df_sous_nut = pd.read_csv("/Users/teilo/Desktop/data/fr_sousalimentation.csv")[sa_columns]
df_sous_nut.rename(columns={'Code zone' : 'code', 'Code Produit' : 'code_produit', 'Valeur' : 'sous_nut', 'Année': 'annee'}, inplace=True)
df_sous_nut = df_sous_nut[['Zone', 'code', 'annee' , 'sous_nut']]
# Application du format année
df_sous_nut['annee'] = df_sous_nut['annee'].apply(format_annee)


# Suppression des doublons et des données <0.1
df_sous_nut['sous_nut'] = pd.to_numeric(df_sous_nut['sous_nut'], errors='coerce')
df_sous_nut['sous_nut'] *= 1000000



# Determination des pays en sous nutrition
df_sous_nut['is_sous_nut'] = False
df_sous_nut = df_sous_nut[df_sous_nut['annee'] == 2013]
df_sous_nut.loc[df_sous_nut['sous_nut'] > 0, 'is_sous_nut'] = True
df_sous_nut.head(10)

,Zone,code,annee,sous_nut,is_sous_nut
0,Afghanistan,2,2013,7900000.0,True
5,Afrique du Sud,202,2013,2600000.0,True
10,Albanie,3,2013,200000.0,True
15,Algérie,4,2013,1700000.0,True
20,Allemagne,79,2013,NaN,False
25,Andorre,6,2013,NaN,False
30,Angola,7,2013,8100000.0,True
35,Antigua-et-Barbuda,8,2013,NaN,False
40,Arabie saoudite,194,2013,1600000.0,True
45,Argentine,9,2013,1500000.0,True


In [18]:
pourcen_sous_nut = df_sous_nut.loc[df_sous_nut['annee'] == 2013, 'sous_nut'].sum()
print('Personnes en sous nutrition en 2013:', pourcen_sous_nut, 'soit', round(pourcen_sous_nut*100/pop_total, 2), '% de la population mondiale')

Personnes en sous nutrition en 2013: 881400000.0 soit 12.6 % de la population mondiale


In [19]:
# Partie 11 ---------------------------------------------------------------------------------------------------------
# Quelle proportion (en termes de poids) est destinée à l'alimentation animale ? (Céréales)

# Détermination des produits "céreales" via is_cereal
df_cereals = pd.read_csv("/Users/teilo/Desktop/data/fr_céréales.csv")
df_cereals.rename(columns={'Code zone' : 'code', 'Code Produit' : 'code_produit'}, inplace=True)
df_cereals = df_cereals[['Produit', 'code_produit']].drop_duplicates()

# ajout 'is_cereal'
main_df['is_cereal'] = main_df['Produit'].isin(df_cereals['Produit'])
cereals = main_df.loc[main_df['is_cereal'], :].copy()

# Calculs des valeurs
manger_humain = cereals['Nourriture'].sum()
manger_animaux = cereals['alim_anim'].sum()
manger_total = manger_humain + manger_animaux
print('Les animaux mangent', round(100*manger_animaux/manger_total), '% des céréales produitent dans le monde')

Les animaux mangent 46 % des céréales produitent dans le monde


In [20]:
# Création de la table pour pays en sous nutrition
pays_sous_nut = df_sous_nut.loc[df_sous_nut['sous_nut'] > 0, "Zone"].unique()

temp = main_df.loc[main_df["Zone"].isin(pays_sous_nut), :]
temp = temp.groupby('Produit').sum().reset_index()
temp = temp.sort_values('quant_exportation', ascending=False)
temp = temp[['Produit', 'code','quant_exportation', 'quant_importation']]

liste_produit = temp['Produit']

temp.head(20)

,Produit,code,quant_exportation,quant_importation
39,Huile de Palme,11909,4.628500e+10,3.178400e+10
52,Maïs,12151,3.788800e+10,6.124700e+10
51,Manioc,11238,3.584300e+10,3.448400e+10
79,Riz (Eq Blanchi),12151,3.278700e+10,2.813200e+10
83,Sucre Eq Brut,12151,2.523700e+10,3.880600e+10
12,Blé,12151,2.484100e+10,1.033040e+11
49,"Légumes, Autres",12151,1.994800e+10,6.114000e+09
8,Bananes,11438,1.780500e+10,3.329000e+09
25,"Fruits, Autres",12151,1.459000e+10,8.416000e+09
48,Lait - Excl Beurre,12151,1.448600e+10,4.255200e+10


In [21]:
# Partie 12 ---------------------------------------------------------------------------------------------------------------
# 3 produits qui ont la plus grande valeur (importation/exportation) pour chacun des 2 ratios(autres/dispo interieur) et (quantités nourritures animale/quantité nourritures) (vous aurez donc 6 produits à citer)

# Création du DF pour les importation
temp_df = main_df.loc[main_df['Produit'].isin(liste_produit), :]
temp_df = temp_df.sort_values('quant_importation', ascending=False)[:200]
temp_df = temp_df[['Produit', 'Nourriture', 'alim_anim', 'autres_usages', 'dispo_interieur']]

#Groupby par produit
produit_df = temp_df.groupby('Produit').sum()

# Création des ratios et implémentation
produit_df['ratio_autres_usages'] = produit_df['autres_usages']/produit_df['dispo_interieur']
produit_df['ratio_alim'] = produit_df['alim_anim']/(produit_df['alim_anim']+produit_df['Nourriture'])

produit_df

,Nourriture,alim_anim,autres_usages,dispo_interieur,ratio_autres_usages,ratio_alim
Produit,,,,,,
"Alcool, non Comestible",0.000000e+00,0.000000e+00,4.780000e+08,4.780000e+08,1.000000,NaN
Ananas,2.032000e+09,0.000000e+00,0.000000e+00,2.089000e+09,0.000000,0.000000
Avoine,1.265000e+09,1.430000e+09,0.000000e+00,2.797000e+09,0.000000,0.530612
Bananes,6.025000e+09,0.000000e+00,0.000000e+00,6.311000e+09,0.000000,0.000000
Bière,2.526100e+10,0.000000e+00,0.000000e+00,2.526100e+10,0.000000,0.000000
Blé,2.901800e+11,9.550300e+10,1.436300e+10,4.403380e+11,0.032618,0.247620
Café,1.926000e+09,0.000000e+00,0.000000e+00,1.926000e+09,0.000000,0.000000
Citrons & Limes,3.004000e+09,0.000000e+00,1.000000e+06,3.071000e+09,0.000326,0.000000
Crustacés,1.763000e+09,0.000000e+00,0.000000e+00,1.763000e+09,0.000000,0.000000


In [22]:
produit_df.sort_values('ratio_autres_usages', ascending=False)[:3]

,Nourriture,alim_anim,autres_usages,dispo_interieur,ratio_autres_usages,ratio_alim
Produit,,,,,,
"Alcool, non Comestible",0.000000e+00,0.0,4.780000e+08,4.780000e+08,1.000000,NaN
Huile de Palme,6.078000e+09,0.0,1.664800e+10,2.284300e+10,0.728801,0.0
Huile de Colza&Moutarde,2.094000e+09,0.0,5.117000e+09,7.211000e+09,0.709610,0.0


In [23]:
produit_df.sort_values('ratio_alim', ascending=False)[:3]

,Nourriture,alim_anim,autres_usages,dispo_interieur,ratio_autres_usages,ratio_alim
Produit,,,,,,
Sorgho,0.0,1.798000e+09,0.0,1.798000e+09,0.000000,1.000000
Graines Colza/Moutarde,56000000.0,4.142000e+09,91000000.0,3.304300e+10,0.002754,0.986660
Orge,338000000.0,2.015300e+10,12000000.0,2.947300e+10,0.000407,0.983505


In [24]:
# Partie 13 ---------------------------------------------------------------------------------------------------------------
# Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

# Création du DF
usa_df = main_df[main_df['Zone'] == "États-Unis d'Amérique"].copy()
usa_df = usa_df[usa_df['Origine'] == "vegetal"].copy()

# Calcul des valeurs
cereal_pour_animaux = usa_df['alim_anim'].sum()
cereal_pour_animaux = cereal_pour_animaux * 0.1 / 1000

print(f"""10% des ressources des USA pour leur production animaliere : {cereal_pour_animaux:,} tonnes""")

10% des ressources des USA pour leur production animaliere : 14,528,900.0 tonnes


In [25]:
# Partie 14 -----------------------------------------------------------------------------------------------------------------------
# En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

# Création du DF
thai_df = main_df[main_df['Zone'] == "Thaïlande"].copy()
thai_df = thai_df[thai_df['Produit'] == "Manioc"].copy()

# Calcul des valeurs
manioc_exporte = thai_df['quant_exportation'].iloc[0]
manioc_produit = thai_df['Production'].iloc[0]

print(f"""La Thaïlande exporte : {manioc_exporte:,}kg de Manioc soit ({manioc_exporte / manioc_produit:.0%}) de leur production.""")


# Calcul population sous alimentés
thai_pop  = main_df[main_df['Zone'] == "Thaïlande"].iloc[0]['pop_total']
thai_sn = df_sous_nut[df_sous_nut['Zone'] == "Thaïlande"]['sous_nut'].iloc[0]


print(f"""Il y a {thai_sn:,} personnes sous alimentés en Thaïlande soit ({thai_sn / thai_pop:.0%}) de leur population.""")


## FIN

La Thaïlande exporte : 25,214,000,000.0kg de Manioc soit (83%) de leur production.
Il y a 5,600,000.0 personnes sous alimentés en Thaïlande soit (8%) de leur population.
